In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marathi-summarization-dataset/marathi-marathi_val.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_train.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_test.jsonl
/kaggle/input/summarisaton-mar-emg/mar_eng_sum.csv
/kaggle/input/english-summarization-dataset/test_final_eng.csv
/kaggle/input/english-summarization-dataset/val_final_eng.csv
/kaggle/input/english-summarization-dataset/train_final_eng.csv


In [3]:
import pandas as pd
import json

# Path to the JSON Lines file
file_path = ["/kaggle/input/marathi-summarization-dataset/marathi-marathi_test.jsonl", "/kaggle/input/marathi-summarization-dataset/marathi-marathi_train.jsonl","/kaggle/input/marathi-summarization-dataset/marathi-marathi_val.jsonl"]


# Initialize an empty list to hold the JSON objects
data = []

# Open the file and read line by line
for i in range(3):
    data.append([])
    with open(file_path[i], 'r', encoding='utf-8') as file:
        for line in file:
            # Parse each line as a JSON object and append to the list
            data[i].append(json.loads(line))
    

# Convert the list of JSON objects into a DataFrame
df_mr = pd.DataFrame(data[1])
df_test_cs_mar = pd.DataFrame(data[0])
df_val_cs_mar = pd.DataFrame(data[2])

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(len(df_mr))
print(len(df_test_cs_mar))
print(len(df_val_cs_mar))

10558
1188
1254


In [4]:
df_en_test=pd.read_csv("/kaggle/input/english-summarization-dataset/test_final_eng.csv")
df_en_test.columns

Index(['source_url', 'target_url', 'text', 'summary'], dtype='object')

In [5]:
import pandas as pd
df=pd.read_csv("/kaggle/input/summarisaton-mar-emg/mar_eng_sum.csv")
df.columns

Index(['url', 'text_mr', 'summary_en', 'summary_mr', 'text_en'], dtype='object')

In [3]:
df['summary_mr'][0], df['text_mr'][0]

('लहान वयात म्हणजे 17व्या वर्षी त्यांना सक्तीने स्कॉटलंडहून पाकिस्तानला नेण्यात आलं. तिथे नेऊन त्यांचं चुलत भावाशी लग्न लावण्यात आलं. काय झालं नक्की?',
 'नायला अगदी लहान वयातच माझं लग्न चुलत भावाशी ठरवण्यात आलं होतं. या निर्णयाबाबत मी नेहमी अस्वस्थ असे. स्कॉटलंडमध्ये राहणाऱ्या 30वर्षीय नायला यांनी बीबीसी स्कॉटलंडशी \'द नाइन\' कार्यक्रमात बोलताना याबद्दल सांगितलं. नायला सांगतात, "भावाशी लग्न होणार आहे हे मला लहानपणापासूनच माहिती होतं. याची आठवण येताच मला कसंतरी होत असे. पाश्चिमात्य संस्कृती अंगवळणी पडून मी त्यासारखं वागेन अशी भीती माझ्या आईवडिलांना वाटत असे. पाश्चिमात्य संस्कृतीपासून मला दूर राखण्याचं काम आपण करत आहोत असं त्यांना वाटे." मीरपूर मुस्लीम कुटुंबात नायला यांचं बालपण व्यतीत झालं. घरचं वातावरण कर्मठ होतं. मी स्वत:चे विचार मुक्तपणे मांडू इच्छित होते. एक वेगळ्याच प्रकारचं आयुष्य मला जगायचं होतं. जेव्हा पाकिस्तानला जावं लागलं 17व्या वर्षी पाकिस्तानला जावं लागल्याची आठवण नायला आजही विसरलेल्या नाहीत. त्याबद्दल त्या विस्ताराने सांगतात, "तू पाप केलं आहेस असं म्हणतच त्यांनी बोलायला सु

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

def generate_relevance_score(model_path, text, summary):
    """
    Generate a relevance score for a given text and summary pair using a fine-tuned MuRIL model.

    Args:
        model_path (str): Path to the fine-tuned MuRIL model.
        text (str): The input text.
        summary (str): The corresponding summary.

    Returns:
        float: The relevance score.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
    model = AutoModelForSequenceClassification.from_pretrained(model_path)

    # Ensure the model is in evaluation mode
    model.eval()

    # Tokenize the text and summary
    inputs = tokenizer(text, summary, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)

    # Assuming the second column of the output represents the "relevance" class
    relevance_score = probabilities[:, 1].item()  # Convert to Python float

    return relevance_score

# Example usage
model_path = "october-sd/MuRIL_relevance"
text = df['text_mr'][1]
summary = df['summary_mr'][1]

relevance_score = generate_relevance_score(model_path, text, summary)
print(f"Relevance Score: {relevance_score:.2f}")

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

Relevance Score: 0.98


In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

def generate_average_relevance_score(model_path, texts, summaries):
    """
    Generate average relevance score for lists of texts and summaries using a fine-tuned MuRIL model.

    Args:
        model_path (str): Path to the fine-tuned MuRIL model.
        texts (list of str): List of input texts.
        summaries (list of str): List of corresponding summaries.

    Returns:
        float: The average relevance score.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
    model = AutoModelForSequenceClassification.from_pretrained(model_path)

    # Ensure the model is in evaluation mode
    model.eval()

    # Initialize the list to store relevance scores
    relevance_scores = []

    # Process each text-summary pair
    for text, summary in zip(texts, summaries):
        # Tokenize the text and summary
        inputs = tokenizer(text, summary, return_tensors="pt", padding=True, truncation=True, max_length=512)

        # Generate predictions
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)

        # Assuming the second column of the output represents the "relevance" class
        relevance_score = probabilities[:, 1].item()  # Convert to Python float
        relevance_scores.append(relevance_score)

    # Calculate the average relevance score
    average_relevance_score = sum(relevance_scores) / len(relevance_scores)
    return average_relevance_score

# Example usage
model_path = "october-sd/MuRIL_relevance"
texts = list(df['text_mr'][0:100])
summaries = list(df['summary_mr'][0:100])

average_relevance_score = generate_average_relevance_score(model_path, texts, summaries)
print(f"Average Relevance Score: {average_relevance_score:.2f}")

Average Relevance Score: 0.98


In [5]:
from transformers import pipeline

# Load the relevance classification pipeline
relevance_classifier = pipeline("text-classification", model="october-sd/MuRIL_relevance", tokenizer="google/muril-base-cased")

def generate_relevance_score(text, summary):
    """
    Generate a relevance score for a given text and summary pair using a fine-tuned MuRIL model pipeline.

    Args:
        text (str): The input text.
        summary (str): The corresponding summary.

    Returns:
        float: The relevance score.
    """
    # Combine text and summary as the input to the pipeline
    input_pair = text + " " + summary  # Adjust this concatenation based on how your model was fine-tuned

    # Generate predictions using the pipeline
    results = relevance_classifier(input_pair)

    # Extract the relevance score
    # The label and score are model-specific, so adjust based on your model's output
    relevance_score = results[0]['score'] if results[0]['label'] == 'LABEL_1' else 1 - results[0]['score']

    return relevance_score

# Example usage
text = df['text_mr'][1]
summary = df['summary_mr'][1]

relevance_score = generate_relevance_score(text, summary)
print(f"Relevance Score: {relevance_score:.2f}")

2024-04-11 11:00:42.277622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 11:00:42.277721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 11:00:42.412978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

Relevance Score: 0.98


In [ ]:
text = df_val_cs_mar['text'][1]
summary = df_val_cs_mar['summary'][1]

relevance_score = generate_relevance_score(text, summary)
print(f"Relevance Score: {relevance_score:.2f}")

In [13]:
def relevance(texts, summaries):
    model_path = "october-sd/MuRIL_relevance"
    relevance_score = generate_average_relevance_score(model_path,texts, summaries)
    return relevance_score

In [14]:
print(relevance(texts, summaries))

0.9845685958862305


In [8]:
from transformers import pipeline
import torch

# Load the pipeline for text classification using the fine-tuned MuRIL model
relevance_classifier = pipeline("text-classification", model="october-sd/MuRIL_relevance", tokenizer="google/muril-base-cased", device=0)  # device=0 to use GPU

def generate_average_relevance_score(texts, summaries):
    """
    Generate average relevance score for a list of text-summary pairs.

    Args:
        texts (list of str): List of input texts.
        summaries (list of str): List of corresponding summaries.

    Returns:
        float: The average relevance score.
    """
    scores = []
    max_length = 512  # Maximum sequence length
    batch_size = 8  # Adjust based on your GPU memory

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]

        # Prepare the batch for processing
        batch_input = []
        for text, summary in zip(batch_texts, batch_summaries):
            # Truncate texts and summaries to half of max_length to accommodate both
            truncated_text = text[:max_length // 2]
            truncated_summary = summary[:max_length // 2]
            batch_input.append(truncated_text + " " + truncated_summary)  # Concatenate text and summary

        # Get relevance scores for the batch
        batch_scores = relevance_classifier(batch_input)
        
        # Extract and store scores
        for score in batch_scores:
            relevance_score = score['score'] if score['label'] == 'LABEL_1' else 1 - score['score']
            scores.append(relevance_score)

    # Calculate the average relevance score
    average_score = sum(scores) / len(scores)
    return average_score

# Example usage
texts = list(df['text_mr'][0:10])
summaries = list(df['summary_mr'][0:10])

average_relevance_score = generate_average_relevance_score(texts, summaries)
print(f"Average Relevance Score: {average_relevance_score:.2f}")

Average Relevance Score: 0.98


In [10]:
texts = list(df_test_cs_mar['text'][0:1000])
summaries = list(df_test_cs_mar['summary'][1:1001])

average_relevance_score = generate_average_relevance_score(texts, summaries)
print(f"Average Relevance Score: {average_relevance_score:.2f}")

Average Relevance Score: 0.06


In [6]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=3ac75085c5e8b0ff8a6834f40048f9fc91d54780a2bc6374542e64360004c838
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [15]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
from langdetect import detect

# Load NER models
tokenizer_en_ner = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

tokenizer_mr_ner = AutoTokenizer.from_pretrained("l3cube-pune/marathi-ner")
model_mr_ner = AutoModelForTokenClassification.from_pretrained("l3cube-pune/marathi-ner")

# Load LaBSE for embeddings
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE")

def detect_language(text):
    try:
        return detect(text)
    except:
        return None

def extract_named_entities(text, tokenizer_ner, model_ner, device='cpu'):
    model_ner.to(device)
    tokens = tokenizer_ner(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model_ner(input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, dim=-1)
    
    named_entities = []
    for token, pred in zip(tokens.tokens(), predictions[0].tolist()):
        if pred != 0:  # Non-entity token
            named_entities.append(token)

    return named_entities

def generate_entity_embeddings(entities, tokenizer_emb, model_emb, device='cpu'):
    model_emb.to(device)
    entity_embeddings = []
    for entity in entities:
        encoded_entity = tokenizer_emb(entity, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            entity_output = model_emb(**encoded_entity)
        entity_embeddings.append(entity_output.last_hidden_state[:, 0, :].squeeze().cpu().numpy())
    return entity_embeddings

def calculate_concept_coverage(summary_entities, text_entities, cross_lingual=False, threshold=0.7):
    coverage_count = 0
    for summary_entity in summary_entities:
        for text_entity in text_entities:
            if cross_lingual:
                similarity = 1 - cosine(summary_entity, text_entity)
                if similarity > threshold:
                    coverage_count += 1
                    break
            else:
                if summary_entity == text_entity:
                    coverage_count += 1
                    break
    coverage = coverage_count / len(summary_entities) if summary_entities else 0
    return coverage

def concept_coverage_pipeline(summary, text, device='cpu'):
    lang_summary = detect_language(summary)
    lang_text = detect_language(text)

    if lang_summary == 'en' and lang_text == 'en':
        entities_summary = extract_named_entities(summary, tokenizer_en_ner, model_en_ner, device)
        entities_text = extract_named_entities(text, tokenizer_en_ner, model_en_ner, device)
        coverage = calculate_concept_coverage(entities_summary, entities_text, cross_lingual=False)
    elif lang_summary == 'mr' and lang_text == 'mr':
        entities_summary = extract_named_entities(summary, tokenizer_mr_ner, model_mr_ner, device)
        entities_text = extract_named_entities(text, tokenizer_mr_ner, model_mr_ner, device)
        coverage = calculate_concept_coverage(entities_summary, entities_text, cross_lingual=False)
    else:
        # Cross-lingual case
        if lang_summary == 'en':
            entities_summary = extract_named_entities(summary, tokenizer_en_ner, model_en_ner, device)
            entities_text = extract_named_entities(text, tokenizer_mr_ner, model_mr_ner, device)
        else:
            entities_summary = extract_named_entities(summary, tokenizer_mr_ner, model_mr_ner, device)
            entities_text = extract_named_entities(text, tokenizer_en_ner, model_en_ner, device)

        # Generate embeddings for cross-lingual comparison
        summary_embeddings = generate_entity_embeddings(entities_summary, tokenizer_labse, model_labse, device)
        text_embeddings = generate_entity_embeddings(entities_text, tokenizer_labse, model_labse, device)
        coverage = calculate_concept_coverage(summary_embeddings, text_embeddings, cross_lingual=True, threshold=0.7)

    return coverage

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cosine
from langdetect import detect

# Load models and tokenizers
tokenizer_en_ner = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer_mr_ner = AutoTokenizer.from_pretrained("l3cube-pune/marathi-ner")
model_mr_ner = AutoModelForTokenClassification.from_pretrained("l3cube-pune/marathi-ner")
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_en_ner.to(device)
model_mr_ner.to(device)
model_labse.to(device)

def detect_languages(texts):
    languages = []
    for text in texts:
        try:
            lang = detect(text)
        except:
            lang = None
        languages.append(lang)
    return languages

def extract_and_embed_entities(texts, tokenizer_ner, model_ner, tokenizer_emb, model_emb):
    inputs = tokenizer_ner(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    model_ner.eval()
    with torch.no_grad():
        outputs = model_ner(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

    entities_batch = []
    for i, prediction in enumerate(predictions):
        tokens = tokenizer_ner.convert_ids_to_tokens(inputs['input_ids'][i])
        entities = [token for token, pred in zip(tokens, prediction.tolist()) if pred != 0]
        entities_batch.append(entities)

    entity_embeddings_batch = []
    for entities in entities_batch:
        if entities:
            encoded_entities = tokenizer_emb(entities, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            model_emb.eval()
            with torch.no_grad():
                outputs_emb = model_emb(**encoded_entities)
            embeddings = outputs_emb.last_hidden_state[:, 0, :].cpu().numpy()
        else:
            embeddings = np.array([])
        entity_embeddings_batch.append(embeddings)
    return entity_embeddings_batch

def calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold=0.7):
    coverage_scores = []
    for summary_embeddings, text_embeddings in zip(entity_embeddings_summary, entity_embeddings_text):
        coverage = 0
        for summary_emb in summary_embeddings:
            for text_emb in text_embeddings:
                if 1 - cosine(summary_emb, text_emb) > threshold:
                    coverage += 1
                    break
        coverage_scores.append(coverage / len(summary_embeddings) if len(summary_embeddings) > 0 else 0)
    return np.mean(coverage_scores)

def concept_coverage_pipeline(texts, summaries):
    languages_texts = detect_languages(texts)
    languages_summaries = detect_languages(summaries)

    batches = defaultdict(lambda: {'texts': [], 'summaries': []})
    for text, summary, lang_text, lang_summary in zip(texts, summaries, languages_texts, languages_summaries):
        batches[(lang_text, lang_summary)]['texts'].append(text)
        batches[(lang_text, lang_summary)]['summaries'].append(summary)

    overall_coverage = []
    for (lang_text, lang_summary), batch in batches.items():
        if lang_text == 'en' and lang_summary == 'en':
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse)
        elif lang_text == 'mr' and lang_summary == 'mr':
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse)
        else:
            if lang_summary == 'en':
                entity_embeddings_summary = e
            # Cross-lingual casextract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse)
            else:
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse)

        coverage = calculate_coverage(entity_embeddings_summary, entity_embeddings_text)
        overall_coverage.append(coverage)

    return np.mean(overall_coverage)

# Example usage with a list of texts and summaries
texts = list(df['text_en'][0:10])
summaries = list(df['summary_en'][0:10])
average_coverage = concept_coverage_pipeline(texts, summaries)
print(f"Average Concept Coverage: {average_coverage}")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average Concept Coverage: 0.9458333333333334


In [67]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cosine
from langdetect import detect

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models and tokenizers
tokenizer_en_ner = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english").to(device)
tokenizer_mr_ner = AutoTokenizer.from_pretrained("l3cube-pune/marathi-ner")
model_mr_ner = AutoModelForTokenClassification.from_pretrained("l3cube-pune/marathi-ner").to(device)
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE").to(device)


BATCH_SIZE = 16

def detect_languages(texts):
    languages = []
    for text in texts:
        try:
            lang = detect(text)
        except:
            lang = None
        languages.append(lang)
    return languages

def extract_and_embed_entities(texts, tokenizer_ner, model_ner, tokenizer_emb, model_emb, device):
    # Initialize lists for batch processing
    entity_embeddings_batch = []

    # Process texts in batches
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        # Tokenization and model inference within torch.no_grad() to save memory
        with torch.no_grad():
            inputs = tokenizer_ner(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model_ner(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

        # Entity extraction and embedding generation also within torch.no_grad()
        with torch.no_grad():
            for i, prediction in enumerate(predictions):
                tokens = tokenizer_ner.convert_ids_to_tokens(inputs['input_ids'][i])
                entities = [token for token, pred in zip(tokens, prediction.tolist()) if pred != 0]

                if entities:
                    encoded_entities = tokenizer_emb(entities, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
                    outputs_emb = model_emb(**encoded_entities)
                    embeddings = outputs_emb.last_hidden_state[:, 0, :].cpu().numpy()
                else:
                    embeddings = np.array([])
                entity_embeddings_batch.append(embeddings)

    return entity_embeddings_batch

def calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold):
    coverage_scores = []
    for summary_embeddings, text_embeddings in zip(entity_embeddings_summary, entity_embeddings_text):
        coverage = 0
        for summary_emb in summary_embeddings:
            for text_emb in text_embeddings:
                if 1 - cosine(summary_emb, text_emb) > threshold:
                    coverage += 1
                    break
        coverage_scores.append(coverage / len(summary_embeddings) if len(summary_embeddings) > 0 else 0)
    return np.mean(coverage_scores)

def concept_coverage_pipeline(texts, summaries):
    languages_texts = detect_languages(texts)
    languages_summaries = detect_languages(summaries)

    batches = defaultdict(lambda: {'texts': [], 'summaries': []})
    for text, summary, lang_text, lang_summary in zip(texts, summaries, languages_texts, languages_summaries):
        batches[(lang_text, lang_summary)]['texts'].append(text)
        batches[(lang_text, lang_summary)]['summaries'].append(summary)

    overall_coverage = []
    for (lang_text, lang_summary), batch in batches.items():
        if lang_text == 'en' and lang_summary == 'en':
            threshold = 0.8
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
        elif lang_text == 'mr' and lang_summary == 'mr':
            threshold = 0.75
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
        else:
            # Cross-lingual case
            if lang_summary == 'en':
                threshold = 0.8
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            else:
                threshold = 0.83
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)

        coverage = calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold)
        overall_coverage.append(coverage)
        
        c = np.mean(overall_coverage)
#         if (lang_text == 'mr' and lang_summary == 'mr') or lang_summary == 'mr':
#             if c > 0.5:
#                 if c < 0.7:
#                     c += 0.2
#             elif c < 0.5:
#                 if c > 0.3:
#                     c -= 0.2
                    
    torch.cuda.empty_cache()
    return c 
   

# Example usage with a list of texts and summaries
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_coverage = concept_coverage_pipeline(texts, summaries)
# print(f"Average Concept Coverage: {average_coverage}")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [73]:
texts = list(df['text_en'][0:100])
summaries = list(df['summary_en'][1:101])
average_coverage = concept_coverage_pipeline(texts, summaries)
print(f"Average Concept Coverage: {average_coverage}")

Average Concept Coverage: 0.21739722434913916


In [19]:
df['summary_mr'][2], df['text_mr'][4]

('जगभरात कोरोनाचा उद्रेक झालेला असताना जगाच्या पाठीवर एक असाही देश आहे ज्याने आपल्याकडे एकालाही कोरोना विषाणूची लागण झालेली नाही, असा दावा केला आहे. हा देश आहे उत्तर कोरिया.',
 'नवीन आकडेवारीनुसार 35.70 कोटींपेक्षा जास्त मुलं युद्धजन्य परिस्थितींमध्ये राहत आहेत. 1990 साली ही संख्या 20 कोटी होती. त्यात आतापर्यंत 75 टक्क्यांनी वाढ झाली आहे. सीरिया, अफगाणिस्तान आणि सोमालिया हे देश मुलांसाठी सर्वाधिक घातक असल्याचं समजतं. मध्य-पूर्व आशियामधील बहुतांश मुलं युद्धजन्य क्षेत्रांमध्ये राहत आहेत. या भागांमध्ये दर पाच पैकी दोन मुलं जीवघेण्या हल्ल्यांपासून केवळ 50 किमी अंतरावर राहतात. आफ्रिका खंड या क्षेत्रांच्या यादीत दुसऱ्या स्थानावर आहे. युद्धजन्य क्षेत्रात राहणाऱ्या मुलांपैकी जवळजवळ निम्मी मुलं, म्हणजे साधारणतः 16.5 कोटी मुलं अतितीव्र संघर्ष क्षेत्रात असल्याचं या अहवालात सांगण्यात आलं आहे. UN ने मुलांवर होणाऱ्या अत्याचारांबाबत हे सहा मुख्य निकष लावले आहेत - \'सेव्ह द चिल्ड्रन\'ने UN आणि इतर संशोधन अहवालांचा आधार घेत त्यांचा अहवाल तयार केला आहे. पण कोणत्याही संघर्षात अडकलेल्या दोन्ही सैन्यांनी ग

In [21]:
coverage_en_en = concept_coverage_pipeline(df['summary_en'][2], df['text_en'][2])

In [22]:
print(coverage_en_en)

1.0


In [34]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load the LaBSE model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model = AutoModel.from_pretrained("setu4993/LaBSE")

In [45]:
from scipy.spatial.distance import cosine

def semantic_similarity(text1, text2, tokenizer, model):
    # Tokenize and encode the texts
    inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate embeddings
    with torch.no_grad():
        embeddings1 = model(**inputs1).pooler_output
        embeddings2 = model(**inputs2).pooler_output

    # Normalize the embeddings
    embeddings1 = torch.nn.functional.normalize(embeddings1)
    embeddings2 = torch.nn.functional.normalize(embeddings2)

    # Compute cosine similarity
    sim = torch.mm(embeddings1, embeddings2.transpose(0, 1))
    
    return sim.item()

In [57]:
original_text = df['text_en'][2]
generated_summary = df['summary_en'][3]
# Calculate the semantic similarity
similarity_score = semantic_similarity(original_text, generated_summary, tokenizer, model)
print(f"Semantic Similarity Score: {similarity_score:.4f}")

Semantic Similarity Score: 0.1565


In [49]:
from transformers import AutoModel, AutoTokenizer
import torch

def sem_sco(text1, text2):
    tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
    model = AutoModel.from_pretrained("setu4993/LaBSE")
    
    similarity_score = semantic_similarity(original_text, generated_summary, tokenizer, model)
    return similarity_score

In [52]:
similarity_score = sem_sco(original_text, generated_summary)
print(f"Semantic Similarity Score: {similarity_score:.4f}")

Semantic Similarity Score: 0.4575


In [66]:
def evaluation_metric(text, summary):
    similarity_score = sem_sco(text, summary)
    coverage = concept_coverage_pipeline(summary, text)
    r = relevance(text, summary)
    
    w1 = 0.3
    w2 = 0.2
    w3 = 0.5
    
    score = similarity_score * w1 + coverage * w2 + r * w3
    return score

In [69]:
s = evaluation_metric(df['text_en'][5], df['summary_en'][5])
print(s)

0.6993585458397866


In [48]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the LaBSE tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
model.eval()  # Set the model to evaluation mode

# Function to calculate embeddings for a batch of texts
def get_embeddings(texts, tokenizer, model, device='cuda'):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Taking mean of the last hidden states to get sentence embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Function to calculate average semantic similarity for lists of texts and summaries
def average_semantic_similarity(texts, summaries, tokenizer, model, batch_size=16):
    similarities = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]
        # Get embeddings
        text_embeddings = get_embeddings(batch_texts, tokenizer, model)
        summary_embeddings = get_embeddings(batch_summaries, tokenizer, model)
        # Compute cosine similarities
        cos_sim = torch.nn.functional.cosine_similarity(text_embeddings, summary_embeddings)
        similarities.extend(cos_sim.cpu().numpy())
    # Calculate the average similarity
    average_similarity = np.mean(similarities)
    if average_similarity > 0.5:
        if average_similarity < 0.7:
            average_similarity += 0.2
    elif average_similarity < 0.5:
        if average_similarity > 0.3:
            average_similarity -= 0.2
    return average_similarity

# Example usage
texts = list(df['text_en'][0:10])
summaries = list(df['summary_en'][0:10])
average_similarity = average_semantic_similarity(texts, summaries, tokenizer, model)
print(f"Average Semantic Similarity: {average_similarity:.4f}")

Average Semantic Similarity: 0.8503


In [54]:
from transformers import AutoModel, AutoTokenizer
import torch

def sem_sco_final(texts, summaries):
    tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
    model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
    model.eval()  # Set the model to evaluation mode
    
    similarity_score = average_semantic_similarity(texts, summaries, tokenizer, model)
    return similarity_score

In [55]:
def evaluation_metric_final(texts, summaries):
    similarity_score = sem_sco_final(texts, summaries)
    coverage = concept_coverage_pipeline(texts, summaries)
    r = generate_average_relevance_score(texts, summaries)
    
    w1 = 0.3
    w2 = 0.2
    w3 = 0.5
    
    score = similarity_score * w1 + coverage * w2 + r * w3
    return score

In [57]:
texts = list(df['text_en'][0:100])
summaries = list(df['summary_en'][0:100])
s = evaluation_metric_final(texts, summaries)
print(s)

0.885896336368875
